# Training the NER pipeline component

The goal of this project is to update the Named Entity Recognition (NER) pipeline component of the language model we've used in a previous project, [POS Tagging, Syntactic Dependency Parsing and NER](https://github.com/j-n-t/natural_language_processing/blob/master/POS%20Tagging%20and%20NER.ipynb), and to improve its performance.

In order to do that, we'll use **spaCy** and the [**INCEpTION annotation tool**](https://inception-project.github.io/).

#### 1. Perform initial imports

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

from spacy.util import minibatch, compounding
from spacy.pipeline import EntityRecognizer

from spacy import displacy

from mediawiki import MediaWiki

import os
import json

from IPython.core.display import HTML

import random

#### 2. Load data

In our previous project, we've seen that the NER pipeline component was not able to associate the proper entity label to the token **Azores, an archipelago of nine islands in the North Atlantic Ocean**.

To correct this, we'll load the **wikipedia page about Azores and use that text to update our NER pipeline component**.

In [2]:
# load wikipedia page about Azores

wikipedia = MediaWiki()
azores = wikipedia.page('Azores - Wikipedia')
# Azores - Wikipedia corresponds to the title tag of the source code for this page

#### 3. Explore data

In [3]:
# url of the wikipedia page about Azores

azores.url

'https://en.wikipedia.org/wiki/Azores'

In [4]:
# fist 2 sentences (out of a maximum of 10) of the wikipedia page about Azores 

wikipedia.summary("Azores - Wikipedia", sentences=2)

'The Azores ( ə-ZORZ, also US:  AY-zorz; Portuguese: Açores [ɐˈsoɾɨʃ]), officially the Autonomous Region of the Azores (Região Autónoma dos Açores), is one of the two autonomous regions of Portugal (along with Madeira). It is an archipelago composed of nine volcanic islands in the Macaronesia region of the North Atlantic Ocean, about 1,360 km (850 mi) west of continental Portugal, about 1,500 km (930 mi) west of Lisbon, in continental Portugal, about 1,500 km (930 mi) northwest of Morocco, and about 2,980 km (1,850 mi) southeast of Newfoundland, Canada.'

In [5]:
# doc with all the content of the wikipedia page about Azores

doc = nlp(azores.content)

In [6]:
# first sentence of the doc container

list(doc.sents)[0]

The Azores ( ə-ZORZ, also US:  AY-zorz; Portuguese: Açores [ɐˈsoɾɨʃ]), officially the Autonomous Region of the Azores (Região Autónoma dos Açores), is one of the two autonomous regions of Portugal (along with Madeira).

In [7]:
# number of sentences in the doc container

len(list(doc.sents))

513

The wikipedia page about Azores has 513 sentences. To accomplish our goal, we'll use the **first 100 sentences** and test the updated NER pipeline component to see if it behaves as expected.

#### 3. Save data to a text file

In [8]:
# select first 100 sentences

azores_100sent = list(doc.sents)[0:100]

In [9]:
# last 5 sentences

azores_100sent[-5:]

[Beginning in 1868, Portugal issued its stamps overprinted with "AÇORES" for use in the islands.,
 Between 1892 and 1906, it also issued separate stamps for the three administrative districts of the time.,
 During the 18th and 19th centuries, Graciosa was host to many prominent figures, including Chateaubriand, the French writer who passed through upon his escape to America during the French revolution;,
 Almeida Garrett, the Portuguese poet who visited an uncle and wrote some poetry while there; and Prince Albert of Monaco, the 19th century oceanographer who led several expeditions in the waters of the Azores.,
 He arrived on his yacht Hirondelle, and visited the furna da caldeira, the noted hot springs grotto.]

In [10]:
# save azores_100sent to a txt file

# output directory
output_dir = './ner_training/text'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(output_dir+'/azores_100sent.txt', 'w', encoding='utf8') as f:
    for sentence in azores_100sent:
        f.write(sentence.text+'\n')

#### 4. Annotate the text

We will now use the INCEpTION annotation tool together with spaCy to perform the annotation of the text we have just saved.

In order to do this, we will need to **set an external recommender for INCEpTION**. Detailed explanations can be found [here](https://inception-project.github.io/example-projects/external-recommender/) and [here](https://github.com/inception-project/external-recommender-spacy).

After launching **INCEpTION** and the **external recommender based on spaCy**, we need to do the following with INCEpTION:

* create new project - set name and description
* in the 'Documents' tab, import our txt file
* in the 'Recommenders' tab, create a new recommender - set name, layer (Named entity), feature (value), tool (Remote classifier) and remote URL (http://localhost:5000/ner)
* in the 'Tagsets' tab, select 'Named Entity tags', and in 'Tagset Details' change the language to 'en' and select the option 'Annotators may add new tags'

We can now return to the main menu by clicking on the top left corner (INCEpTION). We click on 'Annotation' and open the document.

It's now time to start annotating! We select 'Named entity' from the 'Layer' dropdown menu on the right side of the screen and once we double click on a word and select the corresponding value, the external recommendations will appear highlighted in blue above the tokens.

We can now accept or reject the suggested annotations and make new annotations.

<img src="./ner_training/images/azores_inception.jpg" title="Annotating with INCEpTION" />

After finishing the annotation process, we can **export the document and save it in the [CoNLL 2002](https://www.clips.uantwerpen.be/conll2002/ner/) format**.

#### 5. Convert the file

The next step is to **convert this file to spaCy's json format** as documented [here](https://spacy.io/api/cli#convert) and [here](https://github.com/explosion/spaCy/tree/master/examples/training/ner_example_data).

I've created an `out` folder to store the converted files and ran the following command on the command line: `python -m spacy convert azores_100sent.conll out -c ner -s -n 10 -b en`.

With this .json file, we could now **train our NER pipeline component** from the command line following [these guidelines](https://spacy.io/api/cli#train).

#### 6. Get entity offsets and prepare list of training examples

Alternatively, we could also get the entity offsets from this file and prepare a list of training examples. In order to do this, I've created the function `train_examples_converter` to do this conversion.

After copying the.json file to our project's folder, we can now load the file and pass it to the function.

In [11]:
# load json file

json_dir = './ner_training/annot/out'

with open(json_dir+'/azores_100sent.json', encoding='utf8') as json_file:
    train_examples_json = json.load(json_file)

In [12]:
# check file

train_examples_json[0]['paragraphs'][0]['sentences'][0]

{'tokens': [{'orth': 'The', 'tag': '-', 'ner': 'O'},
  {'orth': 'Azores', 'tag': '-', 'ner': 'U-LOC'},
  {'orth': '(', 'tag': '-', 'ner': 'O'},
  {'orth': 'ə-ZORZ', 'tag': '-', 'ner': 'O'},
  {'orth': ',', 'tag': '-', 'ner': 'O'},
  {'orth': 'also', 'tag': '-', 'ner': 'O'},
  {'orth': 'US', 'tag': '-', 'ner': 'U-GPE'},
  {'orth': ':', 'tag': '-', 'ner': 'O'},
  {'orth': 'AY-zorz', 'tag': '-', 'ner': 'O'},
  {'orth': ';', 'tag': '-', 'ner': 'O'},
  {'orth': 'Portuguese', 'tag': '-', 'ner': 'U-NORP'},
  {'orth': ':', 'tag': '-', 'ner': 'O'},
  {'orth': 'Açores', 'tag': '-', 'ner': 'O'},
  {'orth': '[', 'tag': '-', 'ner': 'O'},
  {'orth': 'ɐˈsoɾɨʃ', 'tag': '-', 'ner': 'O'},
  {'orth': ']', 'tag': '-', 'ner': 'O'},
  {'orth': ')', 'tag': '-', 'ner': 'O'},
  {'orth': ',', 'tag': '-', 'ner': 'O'},
  {'orth': 'officially', 'tag': '-', 'ner': 'O'},
  {'orth': 'the', 'tag': '-', 'ner': 'O'},
  {'orth': 'Autonomous', 'tag': '-', 'ner': 'B-LOC'},
  {'orth': 'Region', 'tag': '-', 'ner': 'I-LOC'},


This file follows the [BILUO scheme](https://spacy.io/api/annotation#biluo) for encoding entity annotations.

In [13]:
len(train_examples_json)

8

We grouped every 10 sentences into a document and so we have 8 documents (even though we selected the first 100 sentences, after importing them to INCEpTION we were left with 75 sentences due to different segmentation rules).

Let's get our list of training examples.

In [14]:
def train_examples_converter(json):
    """Returns the converted list of training examples.
    
    Every element of the list is a tuple consisting of two values:
    the text string and a dictionary for the annotations of the corresponding entities. 

    Args:
        json (list): List of training examples in json format to be converted.

    Returns:
        list: List of converted training examples.
    """

    train_examples = []

    for doc_i in range(len(json)): #looping through each document

        for sent_i in range(len(json[doc_i]['paragraphs'][0]['sentences'])): #looping through each sentence

            text=[]
            tags=[]
            entities=[]

            for token in json[doc_i]['paragraphs'][0]['sentences'][sent_i]['tokens']: #looping through each token

                text.append(token['orth'])
                tags.append(token['ner'])
                if token['ner'] != 'O':
                    start=len(' '.join(text))-len(text[-1])
                    end=len(' '.join(text))
                    if token['ner'].startswith('U'):
                        entities.append((start, end, token['ner'][2:]))
                    elif token['ner'].startswith('B'):
                        start_multi = start
                    elif token['ner'].startswith('L'):
                        end_multi = end
                        entities.append((start_multi, end_multi, token['ner'][2:]))

            train_examples.append((' '.join(text), {'entities':entities}))
            
    return train_examples

In [15]:
train_examples = train_examples_converter(train_examples_json)

assert train_examples[0][0][4:10] == 'Azores', "This LOC entity should be Azores."
assert train_examples[0][0][27:29] == 'US', "This GPE entity should be US."
assert train_examples[1][0][62:73] == 'Macaronesia', "This LOC entity should be Macaronesia."
assert train_examples[1][0][226:234] == 'Portugal', "This GPE entity should be Portugal."

HTML('<div class="alert alert-success">Everything seems to be working just fine!</div>')

In [16]:
# first sentence and corresponding entities

train_examples[0]

('The Azores ( ə-ZORZ , also US : AY-zorz ; Portuguese : Açores [ ɐˈsoɾɨʃ ] ) , officially the Autonomous Region of the Azores ( Região Autónoma dos Açores ) , is one of the two autonomous regions of Portugal ( along with Madeira ) .',
 {'entities': [(4, 10, 'LOC'),
   (27, 29, 'GPE'),
   (42, 52, 'NORP'),
   (93, 124, 'LOC'),
   (172, 175, 'CARDINAL'),
   (198, 206, 'GPE'),
   (220, 227, 'LOC')]})

With this list, we can now **update our NER pipeline component**.

#### 7. Update the NER pipeline component

Before starting the training of the NER pipeline component, it is recommended to disable all other pipeline components.

In [17]:
# pipeline components

nlp.pipe_names

['tagger', 'parser', 'ner']

We need to **disable the tagger and the parser**.

In [18]:
nlp.disable_pipes(['tagger', 'parser'])

[('tagger', <spacy.pipeline.pipes.Tagger at 0x1cbb70ab828>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x1cbb9d5e888>)]

Let's check if this worked as expected.

In [19]:
nlp.pipe_names

['ner']

Only the NER pipeline component is active and we can now train it.

In [20]:
# create output directory for the ner pipeline component

if not os.path.exists('./ner_training/ner_pipe'):
    os.makedirs('./ner_training/ner_pipe')

We will follow [these guidelines](https://spacy.io/usage/training#ner) for the training.

In [21]:
optimizer = nlp.entity.create_optimizer()

for i in range(10):
    random.shuffle(train_examples)
    batch_size = compounding(2.0, 4.0, 1.05)
    batches = minibatch(train_examples, size=batch_size)
    for batch in batches:
        sentences, annotations = zip(*batch)
        #print(i, len(sentences), annotations)
        nlp.update(sentences, annotations, drop=0.2, sgd=optimizer)
        
ner = nlp.get_pipe('ner')
ner.to_disk('./ner_training/ner_pipe')

We start by **creating an optimizer**. The `nlp.entity.create_optimizer()` method allows us to update the weights of our model without losing information about the existing set of entity types.

Next, we **iterate over our training examples** creating **random batches of different sizes** with the help of the `minibatch()` and the `compounding()` methods.

We can then **update our model** using the `nlp.update()` method. I've set a **dropout rate** of 0.2 to avoid memorization of the data.

Finally, we **get the component from the pipeline** and **save it to disk**.

#### 8. Evaluate the NER pipeline component

It's now time to evaluate our updated NER pipeline component.

We'll start by **loading our language model disabling the default NER component**.

In [22]:
nlp = spacy.load('en_core_web_sm', disable=['ner'])

We can now **create a new pipeline instance** and then **load the data into it using the component we have saved before**.

In [23]:
ner = EntityRecognizer(nlp.vocab)
ner.from_disk('./ner_training/ner_pipe')

Next, we **add this component to our pipeline**.

In [24]:
nlp.add_pipe(ner, 'azores_ner')

Let's check our pipeline components.

In [25]:
nlp.pipe_names

['tagger', 'parser', 'azores_ner']

We finally have everything we need to test our new NER pipeline component.

We can go back to our example from [POS Tagging, Syntactic Dependency Parsing and NER](https://github.com/j-n-t/natural_language_processing/blob/master/POS%20Tagging%20and%20NER.ipynb) and see if everything works as expected.

In [26]:
doc = nlp("if that between America and Europe is ample, will that between the Continent and the Azores, or Madeira, or the Canaries, or Ireland, be sufficient?")

In [27]:
for ent in doc.ents:
    print(ent.text+': '+ent.label_+' - '+str(spacy.explain(ent.label_)))

America: GPE - Countries, cities, states
Europe: LOC - Non-GPE locations, mountain ranges, bodies of water
Continent: LOC - Non-GPE locations, mountain ranges, bodies of water
Azores: LOC - Non-GPE locations, mountain ranges, bodies of water
Madeira: LOC - Non-GPE locations, mountain ranges, bodies of water
Canaries: LOC - Non-GPE locations, mountain ranges, bodies of water
Ireland: GPE - Countries, cities, states


We can see that the **updated NER pipeline component can now correctly identify Azores**.

Not only that, but since our example sentences also included references to Madeira, both archipelagos are now correctly identified!

Let's check this in a more visual way to finish this project.

In [28]:
displacy.render(doc, style='ent')

In conclusion, we were able to update our NER pipeline component in a way that it correctly identifies the token Azores and the entity associated with it.

In order to do this, we have used the **Wikipedia text about Azores** and **annotated it with INCEpTION, using spaCy as an external recommender**. With this approach, we were able to include training examples containing the token Azores, but also other entities of numerous types that were already correctly recognized by the model. This helped us to prevent the problem of the model "forgetting" how to annotate the latter entities.

After **converting the training examples into the desired format**, we were then able to use them to **retrain our model and update the NER pipeline component**.